## 모듈로 불러다가 Feature값 추출하기 - 4초

## 직접 필요한거 추려서 주피터 셀별로 실행하기 - 2초

## 첫 번째 셀: 클래스와 필요한 라이브러리 정의

In [67]:
# 필요한 라이브러리와 모듈을 임포트합니다.
import os
import cv2
import numpy as np
import torch
import torchvision
from typing import Dict
from models.i3d.i3d_src.i3d_net import I3D  # I3D 모델 구현
from models._base.base_extractor import BaseExtractor  # 기본 특징 추출기 클래스를 상속
from models.raft.raft_src.raft import RAFT, InputPadder
from models.transforms import (Clamp, PILToTensor, ResizeImproved, ScaleTo1_1, TensorCenterCrop, ToFloat, PermuteAndUnsqueeze, ToUInt8)
from utils.io import reencode_video_with_diff_fps  # 동영상 FPS 조정 유틸리티
from utils.utils import dp_state_to_normal, show_predictions_on_dataset
from models.raft.extract_raft import DATASET_to_RAFT_CKPT_PATHS

# I3D 모델을 사용해 RGB 데이터만 추출하는 클래스
class ExtractI3D(BaseExtractor):
    def __init__(self, args) -> None:
        super().__init__(
            feature_type=args['feature_type'],
            on_extraction=args.get('on_extraction', 'save_numpy'),
            tmp_path=args['tmp_path'],
            output_path=args['output_path'],
            keep_tmp_files=args['keep_tmp_files'],
            device=args['device'],
        )
        self.flow_type = args.flow_type
        self.streams = ['rgb',"flow"]
        self.i3d_classes_num = 400
        self.min_side_size = 256
        self.central_crop_size = 224
        self.extraction_fps = args.get('extraction_fps', None)
        self.step_size = args.get('step_size', 16)
        self.stack_size = args.get('stack_size', 16)
        self.resize_transforms = torchvision.transforms.Compose([
            torchvision.transforms.ToPILImage(),
            ResizeImproved(self.min_side_size),
            PILToTensor(),
            ToFloat(),
        ])
        self.i3d_transforms = {
            'rgb': torchvision.transforms.Compose([
                TensorCenterCrop(self.central_crop_size),
                ScaleTo1_1(),
                PermuteAndUnsqueeze()
            ]),
            'flow': torchvision.transforms.Compose([
                TensorCenterCrop(self.central_crop_size),
                Clamp(-20, 20),
                ToUInt8(),
                ScaleTo1_1(),
                PermuteAndUnsqueeze()
            ])
        }
        self.name2module = self.load_model()

    @torch.no_grad()
    def extract(self, video_path: str) -> Dict[str, np.ndarray]:
        if self.extraction_fps is not None:
            video_path = reencode_video_with_diff_fps(video_path, self.tmp_path, self.extraction_fps)

        cap = cv2.VideoCapture(video_path)
        rgb_stack = []
        feats_dict = {'rgb': [],"flow":[]}
        padder = None
        first_frame = True
        while cap.isOpened():
            frame_exists, rgb = cap.read()

            if first_frame:
                first_frame = False
                if not frame_exists:
                    continue

            if frame_exists:
                rgb = cv2.cvtColor(rgb, cv2.COLOR_BGR2RGB)
                rgb = self.resize_transforms(rgb)
                rgb = rgb.unsqueeze(0)
                if self.flow_type == 'raft' and padder is None:
                    padder = InputPadder(rgb.shape)

                rgb_stack.append(rgb)

                if len(rgb_stack) == self.stack_size + 1:
                    batch_feats_dict = self.run_on_a_stack(rgb_stack,padder)
                    for stream in self.streams:
                        feats_dict[stream].extend(batch_feats_dict[stream].tolist())
                    rgb_stack = rgb_stack[self.step_size:]  # 스텝 사이즈만큼 스택에서 제거
            else:
                cap.release()
                break

        if (self.extraction_fps is not None) and (not self.keep_tmp_files):
            os.remove(video_path)

        feats_dict = {stream: np.array(feats) for stream, feats in feats_dict.items()}
        return feats_dict

    def run_on_a_stack(self, rgb_stack, padder=None) -> Dict[str, torch.Tensor]:
        models = self.name2module['model']
        flow_xtr_model = self.name2module.get('flow_xtr_model', None)
        rgb_stack = torch.cat(rgb_stack).to(self.device)

        batch_feats_dict = {}
        for stream in self.streams:
            # if i3d stream is flow, we first need to calculate optical flow, otherwise, we use rgb
            # `end_idx-1` and `start_idx+1` because flow is calculated between f and f+1 frames
            # we also use `end_idx-1` for stream == 'rgb' case: just to make sure the feature length
            # is same regardless of whether only rgb is used or flow
            if stream == 'flow':
                if self.flow_type == 'raft':
                    stream_slice = flow_xtr_model(padder.pad(rgb_stack)[:-1], padder.pad(rgb_stack)[1:])
                else:
                    raise NotImplementedError
            elif stream == 'rgb':
                stream_slice = rgb_stack[:-1]
            else:
                raise NotImplementedError
            # apply transforms depending on the stream (flow or rgb)
            stream_slice = self.i3d_transforms[stream](stream_slice)
            # extract features for a stream
            batch_feats_dict[stream] = models[stream](stream_slice, features=True)  # (B, 1024)
            # add features to the output dict

        return batch_feats_dict

    def load_model(self) -> Dict[str, torch.nn.Module]:
        """Defines the models, loads checkpoints, sends them to the device.
        Since I3D is two-stream, it may load a optical flow extraction model as well.

        Returns:
            Dict[str, torch.nn.Module]: model-agnostic dict holding modules for extraction and show_pred
        """
        flow_model_paths = {'raft': DATASET_to_RAFT_CKPT_PATHS['sintel'], }
        i3d_weights_paths = {
            'rgb': './models/i3d/checkpoints/i3d_rgb.pt',
            'flow': './models/i3d/checkpoints/i3d_flow.pt',
        }
        name2module = {}

        if "flow" in self.streams:
            # Flow extraction module
            if self.flow_type == 'raft':
                flow_xtr_model = RAFT()
            else:
                raise NotImplementedError(f'Flow model {self.flow_type} is not implemented')
            # Preprocess state dict
            state_dict = torch.load(flow_model_paths[self.flow_type], map_location='cpu')
            state_dict = dp_state_to_normal(state_dict)
            flow_xtr_model.load_state_dict(state_dict)
            flow_xtr_model = flow_xtr_model.to(self.device)
            flow_xtr_model.eval()
            name2module['flow_xtr_model'] = flow_xtr_model

        # Feature extraction models (rgb and flow streams)
        i3d_stream_models = {}
        for stream in self.streams:
            i3d_stream_model = I3D(num_classes=self.i3d_classes_num, modality=stream)
            i3d_stream_model.load_state_dict(torch.load(i3d_weights_paths[stream], map_location='cpu'))
            i3d_stream_model = i3d_stream_model.to(self.device)
            i3d_stream_model.eval()
            i3d_stream_models[stream] = i3d_stream_model
        name2module['model'] = i3d_stream_models

        return name2module

## 두 번째 셀: 설정과 인스턴스 생성

In [68]:
# 필요한 설정을 담은 args 딕셔너리를 정의합니다.
# 이 설정은 ExtractI3D 클래스를 사용하기 위해 필요한 정보를 포함합니다.
# 실제 경로 및 설정은 사용 환경에 맞게 조정해야 합니다.
from omegaconf import OmegaConf

args = OmegaConf.create({
    'feature_type': 'i3d',
    'device': 'cuda:0',  # 또는 'cpu'를 사용하시면 됩니다.
    'on_extraction': 'ignore',  # 이 설정은 무시될 것입니다.
    'output_path': 'ignore',  # 이 설정도 무시됩니다.
    'stack_size': 16,  # 이 값들은 예시이며 실제 값으로 대체해야 합니다.
    'step_size': 16,
    'streams': None,
    'flow_type': 'raft',
    'extraction_fps': 25,
    'tmp_path': './tmp/i3d',
    'keep_tmp_files': False,
    'show_pred': False,
    'config': None
    # 여기에 args_cli에 필요한 나머지 설정을 추가하십시오.
})

# ExtractI3D 클래스의 인스턴스를 생성합니다.
# 위에서 정의한 args 딕셔너리를 생성자에 전달합니다.
extractor = ExtractI3D(args)


## 세 번째 셀: 동영상 처리

In [63]:
# 동영상 파일 경로를 지정합니다.
# 이 경로는 실제로 특징을 추출하고자 하는 동영상 파일의 위치를 가리킵니다.
# 사용자 환경에 맞게 해당 경로를 수정해야 합니다.
import time


video_path = '/home/ubuntu/video_features/video_test_0000004.mp4'

# 앞서 생성한 ExtractI3D 인스턴스의 extract 메소드를 호출하여
# 지정된 동영상 파일에서 RGB 특징을 추출합니다.
# 이 메소드는 추출된 특징들을 사전 형태로 반환합니다.
start_time= time.time()
features = extractor.extract(video_path)

# 추출된 RGB 특징을 numpy 배열로 받아서 변수에 저장합니다.
# 'rgb' 키를 사용하여 사전에서 RGB 특징에 접근할 수 있습니다.
rgb_features = features['rgb']
flow_features = features['flow']
concatenated_features = np.concatenate((rgb_features, flow_features), axis=1)
end_time= time.time()

# 선택적: 추출된 특징의 크기나 내용을 확인하기 위해 print 함수를 사용할 수 있습니다.
print(f"추출된 RGB 특징의 크기: {rgb_features.shape}")
print(f"추출된 flow 특징의 크기: {flow_features.shape}")
print(f"추출된 flow 특징의 크기: {concatenated_features.shape}")
np.save('video4.npy', concatenated_features) 
print(end_time-start_time)
print(concatenated_features)

추출된 RGB 특징의 크기: (52, 1024)
추출된 flow 특징의 크기: (52, 1024)
추출된 flow 특징의 크기: (52, 2048)
10.7276771068573
[[0.32722583 1.25116682 0.31964004 ... 0.1165202  0.17485085 0.26154089]
 [0.69086689 0.75427979 0.40393949 ... 0.4031609  0.01892482 0.57530981]
 [0.55582756 0.77099133 0.44683546 ... 0.11136977 0.21072075 0.40609398]
 ...
 [0.25432694 0.46524847 0.12953079 ... 0.20917004 0.00383476 0.36249563]
 [0.42517626 0.41043964 0.16410686 ... 0.11223916 0.17202944 0.56375074]
 [0.11324847 0.55317342 0.16152973 ... 0.43312672 0.25188816 0.13269226]]
